In [1]:
list_features = []
for i in range (0,4):
    for j in range (0,4):
        for k in range (0,4):
            for w in range (0,4):
                list_features.append([i,j,k,w])
#print (np.shape(list_features))

In [2]:
from itertools import groupby
import tensorflow as tf
import random
import torch
import numpy as np
####### READING THE FASTA CHAIN DATA FROM THE FILE ############
def fasta_iter(name):
    fh = open(name)

    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        # drop the ">"
        headerStr = header.__next__()[1:].strip()

        # join all sequence lines to one.
        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

data=fasta_iter("Ecoli_prom.fa.txt")
training_data1=[]
training_data2=[]


######## CONVERTING DNA SEQUENCE INTO ONE HOT ENCODING ##########

for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in range (len(seq)-4):
        temp1 = []
        for j in range (0,4):
            if(seq[i+j]=='A'):
                temp1.append(0)
            if(seq[i+j]=='T'):
                temp1.append(1)
            if(seq[i+j]=='G'):
                temp1.append(2)
            if(seq[i+j]=='C'):
                temp1.append(3)
        temp.append(temp1)
    training_data1.append(temp)
       
    
    
data=fasta_iter("Ecoli_non_prom.fa.txt")
for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in range (len(seq)-4):
        temp1 = []
        for j in range (0,4):
            if(seq[i+j]=='A'):
                temp1.append(0)
            if(seq[i+j]=='T'):
                temp1.append(1)
            if(seq[i+j]=='G'):
                temp1.append(2)
            if(seq[i+j]=='C'):
                temp1.append(3)
        #print (temp1)
        temp.append(temp1)
    training_data2.append(temp)
#training_data
list_freq1 = []
list_freq2 = []

for i in training_data1:
    temp = []
    for j in range (len(list_features)):
        temp.append(0)
    for j in i:
        for k in range (len(list_features)):
            if (list_features[k]==j):
                temp[k] += 1
    list_freq1.append(temp)
    
for i in training_data2:
    temp = []
    for j in range (len(list_features)):
        temp.append(0)
    for j in i:
        for k in range (len(list_features)):
            if (list_features[k]==j):
                temp[k] += 1
    list_freq2.append(temp)    

In [3]:
########### CREATING LABELS FOR THE OUTPUT ###########
labels1=[]
labels2=[]

for i in range(839):
    labels1.append(1)
for i in range(3000):
    labels2.append(0)

#### SHUFFLE THE DATA ########    

comb=list(zip(list_freq1,labels1))
random.shuffle(comb)
list_freq1,labels1=zip(*comb)

comb=list(zip(list_freq2,labels2))
random.shuffle(comb)
list_freq2,labels2=zip(*comb)

print(np.shape(list_freq1))
print(np.shape(labels1))

#training_data=np.reshape(training_data,(3839,77*16))

#print(np.shape(training_data))


(839, 256)
(839,)


In [4]:
####### SPLITTING DATA INTO TRAIN,VALIDATION AND TEST ############
X_train=list()
Y_train=list()
for i in range(649):
    X_train.append(list_freq1[i])
    Y_train.append(labels1[i])
for i in range(2480):
    X_train.append(list_freq2[i])
    Y_train.append(labels2[i])


# X_train.append(list(list_freq1[:649]))
# Y_train.append(list(labels1[:649]))
# X_train.append(list(list_freq2[:2480]))
# Y_train.append(list(labels2[:2480]))

comb=list(zip(X_train,Y_train))
random.shuffle(comb)
X_train,Y_train=zip(*comb)

X_validation=[]
Y_validation=[]

for i in range(649,720):
    X_validation.append(list_freq1[i])
    Y_validation.append(labels1[i])
for i in range(2480,2740):
    X_validation.append(list_freq2[i])
    Y_validation.append(labels2[i])
# X_validation.append(list_freq1[649:720])
# Y_validation.append(labels1[:720])
# X_validation.append(list_freq2[2480:2740])
# Y_validation.append(labels2[2480:2740])

comb=list(zip(X_validation,Y_validation))
random.shuffle(comb)
X_validation,Y_validation=zip(*comb)

X_test=[]
Y_test=[]
for i in range(720,839):
    X_test.append(list_freq1[i])
    Y_test.append(labels1[i])
for i in range(2740,3000):
    X_test.append(list_freq2[i])
    Y_test.append(labels2[i])
# X_test.append(list_freq1[720:])
# Y_test.append(labels1[720:])
# X_test.append(list_freq2[2740:])
# Y_test.append(labels2[2740:])

comb=list(zip(X_test,Y_test))
random.shuffle(comb)
X_test,Y_test=zip(*comb)


# X_train=list(X_train)
# np.shape(X_train)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV


/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.

In [6]:
"""
parameters1 = {
              'min_samples_leaf': [1,2,3], 
              'n_estimators': [32,40]
             }
rfc = RandomForestClassifier(random_state=42)
CV_rfc = GridSearchCV(estimator=rfc,param_grid=parameters1,cv=5)
CV_rfc.fit(X_train,Y_train)
"""

"\nparameters1 = {\n              'min_samples_leaf': [1,2,3], \n              'n_estimators': [32,40]\n             }\nrfc = RandomForestClassifier(random_state=42)\nCV_rfc = GridSearchCV(estimator=rfc,param_grid=parameters1,cv=5)\nCV_rfc.fit(X_train,Y_train)\n"

In [23]:
parameters = {'bootstrap': True,
              'min_samples_leaf':6,
              'n_estimators': 500, 
              'min_samples_split': 10,
              'max_features': None,
              'max_depth': 20,
              'warm_start':True
             }

RF_model = RandomForestClassifier(**parameters)
RF_model.fit(X_train, Y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [24]:
RF_predictions = RF_model.predict(X_train)
score = accuracy_score(Y_train ,RF_predictions)
print(score*100)


96.51645893256632


In [25]:
RF_predictions = RF_model.predict(X_validation)
score = accuracy_score(Y_validation ,RF_predictions)
print(score*100)

89.12386706948641


In [26]:
RF_predictions = RF_model.predict(X_test)
score = accuracy_score(Y_test ,RF_predictions)
print(score*100)

88.91820580474933
